In [1]:
#!pip install ultralytics

https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format

In [7]:
import os
import json
import shutil

def copy_images(coco_json_path, src_image_dir, dest_image_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    images = coco_data['images']

    for image_info in images:
        file_name = image_info['file_name']
        src_file_path = os.path.join(src_image_dir, file_name)
        dest_file_path = os.path.join(dest_image_dir, os.path.basename(file_name))

        os.makedirs(os.path.dirname(dest_file_path), exist_ok=True)

        if os.path.exists(src_file_path):
            shutil.copy(src_file_path, dest_file_path)
        else:
            print(f"File {src_file_path} does not exist.")

# Paths
train_coco_json = '/data/ephemeral/home/data/dataset/train_0_5.json'
val_coco_json = '/data/ephemeral/home/data/dataset/valid_0_5.json'

src_image_dir = '/data/ephemeral/home/data/dataset'
train_dest_image_dir = '/data/ephemeral/home/data/images/train'
val_dest_image_dir = '/data/ephemeral/home/data/images/valid'

# Copy images
copy_images(train_coco_json, src_image_dir, train_dest_image_dir)
copy_images(val_coco_json, src_image_dir, val_dest_image_dir)

In [8]:
# labels 만들기
import os
import json

def convert_coco_to_yolo(coco_json_path, output_dir, image_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    images = {img['id']: img for img in coco_data['images']}
    annotations = coco_data['annotations']
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

    os.makedirs(output_dir, exist_ok=True)

    for ann in annotations:
        image_id = ann['image_id']
        image_info = images[image_id]
        file_name = image_info['file_name']
        width = image_info['width']
        height = image_info['height']

        bbox = ann['bbox']
        x_center = (bbox[0] + bbox[2] / 2) / width
        y_center = (bbox[1] + bbox[3] / 2) / height
        w = bbox[2] / width
        h = bbox[3] / height

        category_id = ann['category_id']
        yolo_annotation = f"{category_id} {x_center} {y_center} {w} {h}\n"

        label_file_name = os.path.splitext(file_name)[0].replace('train/', '') + '.txt'
        label_file_path = os.path.join(output_dir, label_file_name)

        with open(label_file_path, 'a') as label_file:
            label_file.write(yolo_annotation)

# # Paths
# train_coco_json = '/data/ephemeral/home/data/images/train_0_5.json'
# val_coco_json = '/data/ephemeral/home/data/images/valid_0_5.json'

# train_image_dir = '/data/ephemeral/home/data/images/train'
# val_image_dir = '/data/ephemeral/home/data/images/train'

# train_output_dir = '/data/ephemeral/home/data/labels/train'
# val_output_dir = '/data/ephemeral/home/data/labels/val'

# # Ensure output directories exist
# os.makedirs(train_output_dir, exist_ok=True)
# os.makedirs(val_output_dir, exist_ok=True)

# # Convert COCO to YOLO
# convert_coco_to_yolo(train_coco_json, train_output_dir, train_image_dir)
# convert_coco_to_yolo(val_coco_json, val_output_dir, val_image_dir)

# <class_id> <x_center> <y_center> <width> <height>

In [7]:
# Paths
test_coco_json = '/data/ephemeral/home/data/images/test.json'

test_image_dir = '/data/ephemeral/home/data/images/test'

test_output_dir = '/data/ephemeral/home/data/labels/test'

# Ensure output directories exist
os.makedirs(test_output_dir, exist_ok=True)

# Convert COCO to YOLO
convert_coco_to_yolo(test_coco_json, test_output_dir, test_image_dir)

In [1]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO
from dotenv import load_dotenv
import wandb

In [1]:
import os
from ultralytics import YOLO
from dotenv import load_dotenv
import wandb

# Load .env file
load_dotenv()

# W&B login
wandb_api_key = os.getenv("WANDB_API_KEY")
if wandb_api_key:
    wandb.login(key=wandb_api_key)
else:
    raise ValueError("WANDB_API_KEY not found in .env file")

# Load a model
model = YOLO("yolov8x.pt")

# Initialize WandB
wandb.init(project="object-detection", entity="luckyvicky", name="yolov8")

# Add W&B callback for Ultralytics
model.add_callback("on_train_batch_end", lambda trainer: wandb.log({"batch_loss": trainer.loss, "epoch": trainer.epoch}))
# Train the model
train_results = model.train(
    data="/data/ephemeral/home/Sojeong/level2-objectdetection-cv-07/Backbone/Sojeong/1011_ultralytics/coco_dataset.yaml",  # path to COCO dataset YAML
    epochs=100,  # number of training epochs
    patience=30, # early stopping patience
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    amp=False,
    name="exp1",  # 새로운 실행에 대한 이름 지정
    exist_ok=False           # 동일한 이름의 디렉토리가 있을 때 덮어쓰지 않도록 설정
)

# Finish the W&B run
wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: soojeoong43 (luckyvicky). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


Ultralytics 8.3.11 🚀 Python-3.10.13 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/data/ephemeral/home/Sojeong/level2-objectdetection-cv-07/Backbone/Sojeong/1011_ultralytics/coco_dataset.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=exp1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, r

train: Scanning /data/ephemeral/home/data/labels/train... 3943 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3943/3943 [00:04<00:00, 851.10it/s]


train: New cache created: /data/ephemeral/home/data/labels/train.cache


val: Scanning /data/ephemeral/home/data/labels/valid.cache... 940 images, 0 backgrounds, 0 corrupt: 100%|██████████| 940/940 [00:00<?, ?it/s]


Plotting labels to runs/detect/exp1/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 103 weight(decay=0.0), 110 weight(decay=0.0005), 109 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/exp1
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      31.8G      1.057      4.159      1.476        152        640:   0%|          | 0/247 [00:01<?, ?it/s]


TypeError: <lambda>() missing 1 required positional argument: 'logs'

In [4]:
#!pip install -U ultralytics
#!pip install --upgrade ultralytics==8.0.238 wandb

In [ ]:
# # Evaluate model performance on the validation set
# metrics = model.val()

In [ ]:
# # Perform object detection on an image
# results = model("path/to/image.jpg")
# results[0].show()

# # Export the model to ONNX format
# #path = model.export(format="onnx")  # return path to exported model